In [81]:
# Procesa el dataframe para el machine learning.

import os

import pandas as pd
import numpy as np

In [70]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/data_" for x in dirs]
path_socio = path + "Socioeconomicos/"
path_cfe = path + "CFE/"
csv = ".csv"
socio = [ "Datos_demograficos.csv",
    "GLP_2010_2016.csv" ]
cfe = [ "Electrico_residencial_2010_2017.csv",
    "Porcentaje_mensual.csv"]

In [79]:
# Datos actuales.

p = 0
n = 10

data = pd.read_csv(path_data[p] + "0" + csv)

# Agrega las variables procesadas para cada municipio.
for i in range(1, n):
    data_i = pd.read_csv(path_data[p] + str(i) + csv)
    data.iloc[:, i + 11] = data_i.iloc[:,i + 11]
    
    # Para la temperatura media, copia las variables adicionales.
    if i == 3:
        data.loc[ :, ["Tmean_max_2", "M_verano"] ] = (
          data_i.loc[ :, ["Tmean_max_2", "M_verano"] ] )

# Agrega el precio de gas LP.
df_socio = pd.read_csv(path_socio + socio[1])
data.rename({"Año": "Year"}, axis = 1, inplace = True)
for i in data.itertuples():
    data.loc[i[0], "$GLP"] = df_socio.loc[
        df_socio["CVE_ENT"] == i.CVE_ENT, str(i.Year)].values[0]

# Agrega la población y el PIB.    
df_socio = pd.read_csv(path_socio + socio[0])
for i in data.itertuples():
    for k in ["Poblacion", "PIB"]:
        data.loc[i[0], k ] = df_socio.loc[
            df_socio["CVE_INEGI"] == i.CVE_INEGI,
            k + " " + str(i.Year)].values[0]

# Agrega el consumo y la cantidad de usuarios.
df_cfe = pd.read_csv(path_cfe + cfe[0])
for i in data.itertuples():
    # Tipo de tarifa.
    data.loc[i[0], "Tarifa"] = ( df_cfe
        .where(df_cfe["Tarifa"] != "DAC").dropna()
        .loc[df_cfe["CVE_INEGI"] == i.CVE_INEGI,
        "Tarifa"].array[0] )
    for k in ["Consumo", "Usuarios"]:
        for l in ["1*", "DAC"]:
            data.loc[i[0], k + "_" + l ] = df_cfe.loc[
                (df_cfe["CVE_INEGI"] == i.CVE_INEGI)
                & df_cfe["Tarifa"].str.contains(l, regex = True),
                k + " " + str(i.Year)].values[0]

data.rename({"Year": "Año"}, axis = 1, inplace = True)

# PIB per cápita y densidad de población.
data["PCI"] = data["PIB"] / data["Poblacion"]
data["Densidad_poblacion"] = data["Poblacion"] / data["Area"]

# Corrige los meses de inicio de
# verano no contemplados por CFE.
data["M_verano"] = data["M_verano"].astype(int)
data["M_verano"].clip(2, 5, inplace = True)

# Crea un dato por cada mes.
data = data.loc[data.index.repeat(12)]
data["Mes"] = list(range(1, 13)) * 2469 * 7
data.reset_index(drop = True, inplace = True)

# Carga la información con el consumo mensual.
df_cfe = pd.read_csv(path_cfe + cfe[1])

# Calcula el consumo mensual.
for i in data.itertuples():
    data.loc[i[0], ["Consumo_1*", "Consumo_DAC"]] = (
        np.array( [ i[6], i[8] ] ) * df_cfe.loc[
        i.CVE_ENT, f"{i[-2]}-{i[-1]:02d}-01" ] )

# Calcula los meses correspondientes al verano.
data["Verano"] = np.nan
# 6 columnas por cada mes de verano.
m = ["M_" + str(x) for x in range(1, 7)]
# El número de mes correspondiente a cada mes de verano.
data[m[0]] = data["M_verano"] % 12
for i in range(len(m[1:])):
    data[m[i + 1]] = ( data[m[i]] + 1 ) % 12
data[m] = data[m].replace(0, 12)
# Cambiamos el nombre de las columnas de los meses de verano.
dict_m = dict(zip(m, ["Mes"] * 6))
data.rename(columns = dict_m, inplace = True)
# Si el mes es de verano, el valor de Verano es True.
for i in range(1, 7):
    data.iloc[:, -i] = data["Mes"].isin(data.iloc[:, -i])
data["Verano"] = data.iloc[:, -6:].any(axis = 1)
# Quitamos las columnas correspondientes a los meses de verano.
data = data.iloc[:, :-6]

data.to_csv(path_data[p][:-1] 
    + csv, index = False)

data

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Consumo_DAC,Usuarios_DAC,Tarifa,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Año,Mes,Verano
0,1001,1,1,Aguascalientes,Aguascalientes,2.135152e+07,231727.0,1.619597e+06,4251.0,1,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,797010.0,1.170000e+11,24.024788,5,6.833315,146798.659992,NaN,10.151000,1.166359e+05,2010,1,False
1,1001,1,1,Aguascalientes,Aguascalientes,1.959487e+07,231727.0,1.486348e+06,4251.0,1,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,797010.0,1.170000e+11,24.024788,5,6.833315,146798.659992,NaN,10.151000,1.166359e+05,2010,2,False
2,1001,1,1,Aguascalientes,Aguascalientes,2.210239e+07,231727.0,1.676553e+06,4251.0,1,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,797010.0,1.170000e+11,24.024788,5,6.833315,146798.659992,NaN,10.151000,1.166359e+05,2010,3,False
3,1001,1,1,Aguascalientes,Aguascalientes,2.408999e+07,231727.0,1.827321e+06,4251.0,1,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,797010.0,1.170000e+11,24.024788,5,6.833315,146798.659992,NaN,10.151000,1.166359e+05,2010,4,False
4,1001,1,1,Aguascalientes,Aguascalientes,2.487438e+07,231727.0,1.886819e+06,4251.0,1,-102.295872,21.811400,26.810741,11.998749,19.437870,453.183668,-702.796265,470.864685,-18.328812,NaN,395.324070,797010.0,1.170000e+11,24.024788,5,6.833315,146798.659992,NaN,10.151000,1.166359e+05,2010,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207391,2006,2,6,Baja California,San QuintÃ­n,7.103382e+06,41956.0,7.799311e+05,1433.0,1,-114.508064,29.382368,25.585438,15.210447,20.124537,119.414187,-688.155945,765.135315,-101.899223,133.67215,35.259291,123560.0,1.923989e+10,25.887098,5,0.036144,155712.927404,NaN,11.383837,3.418555e+06,2016,8,True
207392,2006,2,6,Baja California,San QuintÃ­n,6.705804e+06,41956.0,7.362782e+05,1433.0,1,-114.508064,29.382368,25.585438,15.210447,20.124537,119.414187,-688.155945,765.135315,-101.899223,133.67215,35.259291,123560.0,1.923989e+10,25.887098,5,0.036144,155712.927404,NaN,11.383837,3.418555e+06,2016,9,True
207393,2006,2,6,Baja California,San QuintÃ­n,5.552655e+06,41956.0,6.096657e+05,1433.0,1,-114.508064,29.382368,25.585438,15.210447,20.124537,119.414187,-688.155945,765.135315,-101.899223,133.67215,35.259291,123560.0,1.923989e+10,25.887098,5,0.036144,155712.927404,NaN,11.383837,3.418555e+06,2016,10,True
207394,2006,2,6,Baja California,San QuintÃ­n,5.009308e+06,41956.0,5.500077e+05,1433.0,1,-114.508064,29.382368,25.585438,15.210447,20.124537,119.414187,-688.155945,765.135315,-101.899223,133.67215,35.259291,123560.0,1.923989e+10,25.887098,5,0.036144,155712.927404,NaN,11.383837,3.418555e+06,2016,11,False


In [84]:
# Archivo actual.
data = pd.read_csv(path_data[0][:-1] + csv)

# Iteramos para cada RCP.
for p in range(2, 3):
#for p in range(1, 3):

    # Iteramos para cada periodo futuro.
    for f in fut:

        # Calculamos el crecimiento de población y de PIB futuro.
        if not os.path.exists( path_data[p] +
            str(i) + "." + str(f) + csv ):

            for i in [10, 11]:
                # Modelo, periodo actual.
                data_i_0 = pd.read_csv(path_data[p] +
                    str(i) + ".2010" + csv)
                # Modelo, periodo futuro.
                data_i = pd.read_csv(path_data[p] +
                    str(i) + "." + str(f) + csv)
                # Guardamos una copia del archivo original.
                data_i.to_csv(path_data[p] + str(i) + "." +
                     str(f) + ".original" + csv, index = False)
                
                # Porcentaje de cambio.
                data_i.iloc[:,i + 13] = ( data_i.iloc[:,i + 13]
                    / data_i_0.iloc[:,i + 13] )
                data_i.to_csv(path_data[p] + str(i) + 
                    "." + str(f) + "_perc" + csv, index = False)
                
                # Valor proyectado.
                data_i.iloc[:,i + 13] = ( data.iloc[:,i + 13]
                    * data_i.iloc[:,i + 13] )
                data_i.to_csv(path_data[p] + str(i) + 
                    "." + str(f) + csv, index = False)

In [91]:
# Datos futuros.

# Tarifas.
tar = ["1", "1A", "1A", "1A", "1B",
    "1B", "1C", "1D", "1E", "1F"]

# Iteramos para cada RCP.
for p in range(2, 3):
# for p in range(1, 3):
    
    # Iteramos para cada periodo futuro.
    for f in fut:

        data = pd.read_csv(path_data[p]
            + "0." + str(f) + csv)

        n = 12

        # Agrega las variables procesadas para cada municipio.
        for i in range(1, n):
            data_i = pd.read_csv(path_data[p] +
                str(i) + "." + str(f) + csv)
            data.iloc[:, i + 11 ] = data_i.iloc[:, i + 11 ]
            
            # Para la temperatura media, copia las variables adicionales.
            if i == 3:
                data.loc[ :, ["Tmean_max_2", "M_verano"] ] = (
                    data_i.loc[ :, ["Tmean_max_2", "M_verano"] ] )

        # Agrega el precio de gas LP.
        df_socio = pd.read_csv(path_socio + socio[1])
        for i in data.itertuples():
            data.loc[i[0], "$GLP"] = df_socio.loc[
                df_socio["CVE_ENT"] == i.CVE_ENT,
                str(2010 )].values[0]

        # PIB per cápita y densidad de población.
        data["PCI"] = data["PIB"] / data["Poblacion"]
        data["Densidad_poblacion"] = ( 
            data["Poblacion"] / data["Area"] )

        # Convierte la temperatura media máxima en el tipo de tarifa.
        data["Tarifa"] = ( ( data["Tmean_max_2"].apply(np.floor)
            .clip(24, 33) - 24 ).astype(int)
            .apply(lambda x: tar[x]) )

        # Corrige los meses de inicio de
        # verano no contemplados por CFE.
        data["M_verano"] = data["M_verano"].astype(int)
        data["M_verano"].clip(2, 5, inplace = True)

        # Crea un dato por cada mes.
        data = data.loc[data.index.repeat(12)]
        data["Mes"] = list(range(1, 13)) * 2469
        data.reset_index(drop = True, inplace = True)

        # Calcula los meses correspondientes al verano.
        data["Verano"] = np.nan
        # 6 columnas por cada mes de verano.
        m = ["M_" + str(x) for x in range(1, 7)]
        # El número de mes correspondiente a cada mes de verano.
        data[m[0]] = data["M_verano"] % 12
        for i in range(len(m[1:])):
            data[m[i + 1]] = ( data[m[i]] + 1 ) % 12
        data[m] = data[m].replace(0, 12)
        # Cambiamos el nombre de las columnas de los meses de verano.
        dict_m = dict(zip(m, ["Mes"] * 6))
        data.rename(columns = dict_m, inplace = True)
        # Si el mes es de verano, el valor de Verano es True.
        for i in range(1, 7):
            data.iloc[:, -i] = data["Mes"].isin(data.iloc[:, -i])
        data["Verano"] = data.iloc[:, -6:].any(axis = 1)
        # Quitamos las columnas correspondientes a los meses de verano.
        data = data.iloc[:, :-6]

        data.to_csv(path_data[p][:-1] + "."
            + str(f) + csv, index = False)